In [14]:
import cv2
import numpy as np
import os

# Function 1: Smallest Circle that Encapsulates the Particle
def find_encapsulating_circle(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        (x, y), radius = cv2.minEnclosingCircle(largest_contour)
        center = (int(x), int(y))
        radius = int(radius)
        result_image = input_image.copy()
        cv2.circle(result_image, center, radius, (0, 255, 0), 2)
        return result_image
    else:
        return input_image

# Function 2: Total Surface Area of the Particle
def calculate_surface_area(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        particle_area = cv2.contourArea(largest_contour)
        result_image = input_image.copy()
        cv2.putText(result_image, f'Area: {int(particle_area)} pixels^2', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        return result_image
    else:
        return input_image

# Function 3: Major Axis Inside the Particle
def find_major_axis(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        ellipse = cv2.fitEllipse(largest_contour)
        major_axis_length = max(ellipse[1])
        minor_axis_length = min(ellipse[1])
        angle = ellipse[2]
        result_image = input_image.copy()
        center = (int(ellipse[0][0]), int(ellipse[0][1]))
        major_axis_endpoint1 = (
            int(center[0] + major_axis_length / 2 * np.cos(np.radians(angle))),
            int(center[1] + major_axis_length / 2 * np.sin(np.radians(angle)))
        )
        major_axis_endpoint2 = (
            int(center[0] - major_axis_length / 2 * np.cos(np.radians(angle))),
            int(center[1] - major_axis_length / 2 * np.sin(np.radians(angle)))
        )
        cv2.ellipse(result_image, center, (int(major_axis_length / 2), int(minor_axis_length / 2)), angle, 0, 360, (0, 255, 0), 2)
        cv2.line(result_image, major_axis_endpoint1, major_axis_endpoint2, (0, 255, 0), 2)
        return result_image
    else:
        return input_image

# Function 4: Total Perimeter of the Particle
def calculate_perimeter(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        perimeter = cv2.arcLength(largest_contour, True)
        result_image = input_image.copy()
        cv2.drawContours(result_image, [largest_contour], -1, (0, 255, 0), 2)
        cv2.putText(result_image, f'Perimeter: {int(perimeter)} pixels', (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        return result_image
    else:
        return input_image

# Function 5: Centroid of the Particle
def find_centroid(input_image):
    gray = cv2.cvtColor(input_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        largest_contour = max(contours, key=cv2.contourArea)
        M = cv2.moments(largest_contour)
        centroid_x = int(M['m10'] / M['m00'])
        centroid_y = int(M['m01'] / M['m00'])
        result_image = input_image.copy()
        cv2.circle(result_image, (centroid_x, centroid_y), 5, (0, 255, 0), -1)
        return result_image
    else:
        return input_image

def process_and_save_images(input_folder, output_folder):
    # Iterate over the files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpeg"):
            input_path = os.path.join(input_folder, filename)
            input_image = cv2.imread(input_path)

            # Apply the five operations to the input image
            encapsulating_circle = find_encapsulating_circle(input_image)
            surface_area = calculate_surface_area(input_image)
            major_axis = find_major_axis(input_image)
            perimeter = calculate_perimeter(input_image)
            centroid = find_centroid(input_image)

            # Create a folder in the output directory for each image
            image_output_folder = os.path.join(output_folder, os.path.splitext(filename)[0])
            os.makedirs(image_output_folder, exist_ok=True)

            # Save the processed images to the respective output folder
            cv2.imwrite(os.path.join(image_output_folder, 'encapsulating_circle.jpg'), encapsulating_circle)
            cv2.imwrite(os.path.join(image_output_folder, 'surface_area.jpg'), surface_area)
            cv2.imwrite(os.path.join(image_output_folder, 'major_axis.jpg'), major_axis)
            cv2.imwrite(os.path.join(image_output_folder, 'perimeter.jpg'), perimeter)
            cv2.imwrite(os.path.join(image_output_folder, 'centroid.jpg'), centroid)

if __name__ == "__main__":
    input_folder = '/Users/ramavathjagadeesh/Desktop/Project Roop/input_images'  # Input folder containing images
    output_folder = '/Users/ramavathjagadeesh/Desktop/Project Roop/output_images'  # Output folder for saving processed images

    process_and_save_images(input_folder, output_folder)
